In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2

import sys
sys.path.insert(0,"../Classes")
from ScorerClass import scorer

In [2]:
def depth_scale(X):
    bsum = np.sum(X,axis=1)
    bsum[bsum==0] = 1
    out = X/np.moveaxis([bsum]*X.shape[1],(0,1),(1,0))
    return out

In [3]:
def std_scaler(X,is_d=False):
    if is_d:
        bsum = np.sum(X,axis=1)
        bsum[bsum==0] = 1
        out = X/np.moveaxis([bsum]*X.shape[1],(0,1),(1,0))
        return out
    mu = np.mean(X,axis=0)
    sig = np.sqrt(np.var(X,axis=0))
    if any(sig == 0):
        return X
    out = (X-mu)/sig
    return out

In [4]:
from sklearn.linear_model import LogisticRegression
def select_features(Xin,is_d,y,tr,t,p = 0.1):
    X = std_scaler(Xin,is_d)
    clf = LogisticRegression(max_iter = 1000)
    clf.fit(X[tr],y[tr])
    
    #Sometimes problems?
    sdv = np.std(X,axis=0)
    sdv[sdv==0] = 1
    
    importances = clf.coef_/sdv
    truth_val = importances>=np.percentile(importances,100-100*p)
    truth_val = truth_val[0]
    return truth_val

def load_and_select(path,is_d,y,tr,t,p=0.1):
    X = np.load(path)
    fs = select_features(X,is_d,y,tr,t,p)
    X2 = X[:,fs]
    if is_d:
        X2 = depth_scale(X2)
    return X2

In [5]:
def get_data(autoenc=False,rgb=True,lab=True,depth=True):
    Xlist = []
    if autoenc:
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs01.npy",False,y,tr,t))
    if rgb:
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs03.npy",False,y,tr,t))
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs04.npy",False,y,tr,t))
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs05.npy",False,y,tr,t))
    if lab:
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs08.npy",False,y,tr,t))
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs09.npy",False,y,tr,t))
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs10.npy",False,y,tr,t))
    if depth:
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs00.npy",True,y,tr,t))#d
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs02.npy",True,y,tr,t))#d
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs06.npy",True,y,tr,t))#d
        Xlist.append(load_and_select("../../../Data/Sampled_Features/fs07.npy",True,y,tr,t))#d
    
    data = np.hstack(Xlist)
    return data

In [6]:
from sklearn.model_selection import train_test_split
seed = 2666

y    = np.load("../../../Data/Sampled_Features/y.npy")

tr,t = train_test_split([a for a in range(y.shape[0])],random_state=seed)

In [10]:
X_rgb = get_data(rgb=True,lab=False,depth=False)
X_lab = get_data(rgb=False,lab=True,depth=False)
#X_dep = get_data(rgb=False,lab=False,depth=True)[0]
#X_ae  = load_and_select("../../../Data/Sampled_Features/fs01.npy",False,y,tr,t)
#Xf = np.hstack((X_rgb,X_lab,X_dep))
X2 = np.hstack((X_rgb,X_lab))

/home/danielh/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/home/danielh/.local/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:763: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regressi

In [11]:
def do_score(clf,X,y):
    pred = clf.predict(X)
    scorer_ = scorer()
    print("Acc:",scorer_.acc(y,pred))
    print("Sens:",scorer_.sens(y,pred))
    print("Spec:",scorer_.spec(y,pred))
    print("VOI:",scorer_.VOI(y,pred))
    print("GCE",scorer_.GCE(y,pred))

In [12]:
c5 = LogisticRegression(max_iter=10000).fit(X2[tr],y[tr])
print("COL")
do_score(c5,X2[t],y[t])

COL
Acc: 0.6531206751054852
Sens: 0.7103559347248184
Spec: 0.5906469080717014
VOI: 1.2830791494693754
GCE 1.0002705981135027


In [16]:
from sklearn.linear_model import LogisticRegression
c1 = LogisticRegression(max_iter=10000).fit(X_rgb[tr],y[tr])
print("RGB")
do_score(c1,X_rgb[t],y[t])

c2 = LogisticRegression(max_iter=10000).fit(X_lab[tr],y[tr])
print("LAB")
do_score(c2,X_lab[t],y[t])

c3 = LogisticRegression(max_iter=10000).fit(X_dep[tr],y[tr])
print("Dep")
do_score(c3,X_dep[t],y[t])

c4 = LogisticRegression(max_iter=10000).fit(Xf[tr],y[tr])
print("Full")
do_score(c4,Xf[t],y[t])

c5 = LogisticRegression(max_iter=10000).fit(X_ae[tr],y[tr])
print("AE")
do_score(c5,X_ae[t],y[t])

Xfae = np.hstack((Xf,X_ae))
c6 = LogisticRegression(max_iter=10000).fit(Xfae[tr],y[tr])
print("FullAE")
do_score(c6,Xfae[t],y[t])

RGB
Acc: 0.6567459915611814
Sens: 0.7128590555408229
Spec: 0.5954971301087947
VOI: 1.2787036943083878
GCE 1.0002543966571447
LAB
Acc: 0.6438244725738397
Sens: 0.7087259955888155
Spec: 0.5729827806527679
VOI: 1.292325239310396
GCE 1.000083333938032
Dep
Acc: 0.5218835443037975
Sens: 1.0
Spec: 7.0600029652012455e-06
VOI: 0.6922259229137895
GCE 1.000003375503595
Full
Acc: 0.6587848101265823
Sens: 0.7170567956172748
Spec: 0.5951794299753606
VOI: 1.275419637439503
GCE 1.0000664467322997
AE
Acc: 0.5280168776371308
Sens: 0.7923315244458531
Spec: 0.23951060059445226
VOI: 1.2212749700872343
GCE 0.9954321073264536
FullAE
Acc: 0.6604894514767933
Sens: 0.7168821592812744
Spec: 0.5989353515528476
VOI: 1.2736693584573522
GCE 1.0001411315019615
